In [1]:
import sys
!{sys.executable} -m pip install -r requirements.txt

     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     ---------------------------------------- 60.8/60.8 kB 1.6 MB/s eta 0:00:00
     ---------------------------------------- 0.0/48.7 kB ? eta -:--:--
     ---------------------------------------- 48.7/48.7 kB 2.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
   ---------------------------------------- 0.3/38.8 MB 5.2 MB/s eta 0:00:08
    --------------------------------------- 0.7/38.8 MB 7.3 MB/s eta 0:00:06
    --------------------------------------- 0.8/38.8 MB 8.4 MB/s eta 0:00:05
   - -------------------------------------- 1.4/38.8 MB 7.2 MB/s eta 0:00:06
   - -------------------------------------- 1.7/38.8 MB 7.4 MB/s eta 0:00:06
   -- ------------------------------------- 2.2/38.8 MB 7.7 MB/s eta 0:00:05
   -- ------------------------------------- 2.6/38.8 MB 7.8 MB/s eta 0:00:05
   --- ------------------------------------ 3.0/38.8 MB 7.9 MB/s eta 0:00:05
   --- ---

ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'C:\\Users\\PC1\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python311\\site-packages\\tensorflow\\include\\external\\com_github_grpc_grpc\\src\\core\\ext\\filters\\client_channel\\lb_policy\\grpclb\\client_load_reporting_filter.h'
HINT: This error might have occurred since this system does not have Windows Long Path support enabled. You can find information on how to enable this at https://pip.pypa.io/warnings/enable-long-paths


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\PC1\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
import os
import cv2
import re
import numpy as np
from collections import Counter
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, precision_score
from sklearn.utils import shuffle

In [4]:
dataset_path = "histology_slides"  
regex_label = r"[A-Z]+_([A-Z])_[A-Z]+-\d{2}-[A-Z\d]+-(\d+)-\d+\.png"

In [5]:
def load_data(dataset_path, img_size=(128,128)):
    X = []
    y = []
    num_files = 0

    for root, dirs, files in os.walk(dataset_path):
        if os.path.basename(root) == "400X": 
            for file in files:
                if file.endswith('.png'):
                    img_path = os.path.join(root, file)
                    img = cv2.imread(img_path)
                    img = cv2.resize(img, img_size)
                    X.append(img)

                    # Assuming label is B(enigno) or M(aligno)
                    match_obj = re.search(regex_label, file)
                    label = match_obj.group(1)  # B or M
                    y.append(True if label == "M" else False)

                    num_files += 1

    print(f"Total images processed: {num_files}")

    X = np.array(X)
    y = np.array(y)
    return X, y

In [6]:
img_size = (700, 460)
X, y = load_data(dataset_path, img_size)

Total images processed: 1820


In [5]:


count = Counter(y)
print(count)

Counter({True: 1232, False: 588})


In [7]:
X = X / 255

In [8]:
X, y = shuffle(X, y)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
def build_cnn(input_shape):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(len(np.unique(y)), activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

In [11]:
cnn_model = build_cnn(X_train.shape[1:])
cnn_model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

C:\Users\PC1\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 164s 3s/step - accuracy: 0.6466 - loss: 3.7675 - val_accuracy: 0.6538 - val_loss: 0.7882
Epoch 2/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 138s 3s/step - accuracy: 0.6780 - loss: 0.5917 - val_accuracy: 0.6538 - val_loss: 0.5258
Epoch 3/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 138s 3s/step - accuracy: 0.7557 - loss: 0.4877 - val_accuracy: 0.7857 - val_loss: 0.4589
Epoch 4/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 137s 3s/step - accuracy: 0.8209 - loss: 0.4325 - val_accuracy: 0.8599 - val_loss: 0.3934
Epoch 5/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 137s 3s/step - accuracy: 0.8533 - loss: 0.3822 - val_accuracy: 0.8434 - val_loss: 0.3853
Epoch 6/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 139s 3s/step - accuracy: 0.8605 - loss: 0.3460 - val_accuracy: 0.8571 - val_loss: 0.3912
Epoch 7/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 142s 3s/step - accuracy: 0.8458 - loss: 0.4149 - val_accuracy: 0.8626 - val_loss: 0.3816
Epoch 8/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 152s 3s/step - accuracy: 0.8554 - loss: 0.3522 - val_accuracy: 0.8049 - v

In [12]:
cnn_loss, cnn_accuracy = cnn_model.evaluate(X_test, y_test)
cnn_predictions = np.argmax(cnn_model.predict(X_test), axis=-1)
cnn_precision = precision_score(y_test, cnn_predictions, average='weighted')

12/12 ━━━━━━━━━━━━━━━━━━━━ 7s 546ms/step - accuracy: 0.8528 - loss: 0.4735
12/12 ━━━━━━━━━━━━━━━━━━━━ 7s 540ms/step


In [13]:
X_train_flat = X_train.reshape(X_train.shape[0], -1)
X_test_flat = X_test.reshape(X_test.shape[0], -1)

In [14]:
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train_flat, y_train)
knn_predictions = knn.predict(X_test_flat)
knn_accuracy = accuracy_score(y_test, knn_predictions)
knn_precision = precision_score(y_test, knn_predictions, average='weighted')

In [15]:
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train_flat, y_train)
rf_predictions = rf.predict(X_test_flat)
rf_accuracy = accuracy_score(y_test, rf_predictions)
rf_precision = precision_score(y_test, rf_predictions, average='weighted')

In [16]:
lr = LogisticRegression(max_iter=1000, random_state=42)
lr.fit(X_train_flat, y_train)
lr_predictions = lr.predict(X_test_flat)
lr_accuracy = accuracy_score(y_test, lr_predictions)
lr_precision = precision_score(y_test, lr_predictions, average='weighted')

In [17]:
nb = GaussianNB()
nb.fit(X_train_flat, y_train)
nb_predictions = nb.predict(X_test_flat)
nb_accuracy = accuracy_score(y_test, nb_predictions)
nb_precision = precision_score(y_test, nb_predictions, average='weighted')

In [18]:
print(f"CNN - Accuracy: {cnn_accuracy:.4f}, Precision: {cnn_precision:.4f}")
print(f"KNN - Accuracy: {knn_accuracy:.4f}, Precision: {knn_precision:.4f}")
print(f"Random Forest - Accuracy: {rf_accuracy:.4f}, Precision: {rf_precision:.4f}")
print(f"Logistic Regression - Accuracy: {lr_accuracy:.4f}, Precision: {lr_precision:.4f}")
print(f"Naive Bayes - Accuracy: {nb_accuracy:.4f}, Precision: {nb_precision:.4f}")

CNN - Accuracy: 0.8462, Precision: 0.8475
KNN - Accuracy: 0.7720, Precision: 0.7813
Random Forest - Accuracy: 0.8324, Precision: 0.8356
Logistic Regression - Accuracy: 0.7418, Precision: 0.7340
Naive Bayes - Accuracy: 0.7335, Precision: 0.7772
